In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import numpy as np
from labcore.measurement import *

Problem: we have an instrument that returns *delayed* data, i.e., we write a function that starts an experiment on the instrument but does not return the data.
The data needs to be gathered from the instrument independently afterwards and asynchronously. 
How to incorporate that into our measurement framework?

In [3]:
global_start = None
global_stop = None
global_delay = None


def gather_data(*specs, batchsize=1):
    """A generator that simulates collecting data.
    yields for each spec a different power of the current number.
    """
    global global_start
    global global_stop
    global global_delay
    
    nspecs = len(specs)

    buffer = []
    
    for x in np.arange(global_start, global_stop):
        time.sleep(global_delay)
        buffer.append(tuple([x**i for i in range(1, nspecs+1)]))
        
        if len(buffer) >= batchsize:
            print(f'releasing {len(buffer)} records.')
            for b in buffer:
                yield b
            buffer = []
    
    print(f'releasing {len(buffer)} records.')
    for b in buffer:
        yield b


def delayed_recording(*specs):
    """
    returns a decorator. the decorated function will, when called,
    return a sweep that calls the decorated function once, followed by a Sweep of `gather_data`.
    The output of `gather_data` is recorded as specified by the specs that
    are passed to this function to create the generator.
    """
    
    def decorator(fun):
        
        def sweep(batchsize=1):
            """Return a sweep that executes the decorated setup function and then iterates over
            gathered data."""
            setup_sweep = once(fun)
            gather_sweep = Sweep(record_as(gather_data(*specs, batchsize=batchsize), *specs))
            return setup_sweep + gather_sweep
        
        return sweep
        
    return decorator


# Note: this decoration will give us a function that creates a sweep when called.
# we call it when creating the sweep to avoid the sweep getting exhausted.
@delayed_recording(
    indep('a'), dep('b'), dep('c')
)
def start_my_experiment(start=1, stop=10, delay=0.1):
    global global_start
    global global_stop
    global global_delay
    
    print(f'my parameters: start={start}, stop={stop}, delay={delay}')
    
    global_start = start
    global_stop = stop
    global_delay = delay

In [4]:
help(start_my_experiment)

Help on function sweep in module __main__:

sweep(batchsize=1)
    Return a sweep that executes the decorated setup function and then iterates over
    gathered data.



In [13]:
sweep = start_my_experiment(batchsize=4)

sweep.set_action_opts(
    start_my_experiment=dict(delay=5, start=0, stop=20)
)

for ret in sweep:
    print(ret)

my parameters: start=0, stop=20, delay=5
{'a': None, 'b': None, 'c': None}
releasing 4 records.
{'a': 0, 'b': 0, 'c': 0}
{'a': 1, 'b': 1, 'c': 1}
{'a': 2, 'b': 4, 'c': 8}
{'a': 3, 'b': 9, 'c': 27}
releasing 4 records.
{'a': 4, 'b': 16, 'c': 64}
{'a': 5, 'b': 25, 'c': 125}
{'a': 6, 'b': 36, 'c': 216}
{'a': 7, 'b': 49, 'c': 343}
releasing 4 records.
{'a': 8, 'b': 64, 'c': 512}
{'a': 9, 'b': 81, 'c': 729}
{'a': 10, 'b': 100, 'c': 1000}
{'a': 11, 'b': 121, 'c': 1331}
releasing 4 records.
{'a': 12, 'b': 144, 'c': 1728}
{'a': 13, 'b': 169, 'c': 2197}
{'a': 14, 'b': 196, 'c': 2744}
{'a': 15, 'b': 225, 'c': 3375}
releasing 4 records.
{'a': 16, 'b': 256, 'c': 4096}
{'a': 17, 'b': 289, 'c': 4913}
{'a': 18, 'b': 324, 'c': 5832}
{'a': 19, 'b': 361, 'c': 6859}
releasing 0 records.
